In [2]:
%reset -sf

In [317]:
# LOAD THIS BEFORE START, run rmdl

#!/usr/bin/env python3
import sys
import math, random
import functools, itertools, collections, heapq, bisect
from collections import Counter, defaultdict, deque

# import numpy as np
import scipy
import matplotlib.pyplot as plt

abc = "abcdefghijklmnopqrstuvwxyz"
abc_map = {c:i for i,c in enumerate(abc)}
M9 = 10**9 + 7  # 998244353
yes, no = "YES", "NO"
d4 = [(1,0),(0,1),(-1,0),(0,-1)]
d8 = [
    (-1,-1),(-1,0),(-1,1),
    (0,-1),(0,0),(0,1),
    (1,-1),(1,0),(1,1)]
d6 = [(2,0),(1,1),(-1,1),(-2,0),(-1,-1),(1,-1)]  # hexagonal layout
MAXINT = sys.maxsize

In [55]:
with open("/Users/htong/Downloads/input.txt") as f:
    srr = f.read().strip().split("\n")

In [54]:
srr = """
O....#....
O.OO#....#
.....##...
OO.#O....O
.O.....O#.
O.#..O.#.#
..O..#O..O
.......O..
#....###..
#OO..#....
""".strip().split("\n")

In [56]:
arr = [list(row) for row in srr]

In [59]:
def rotate_90_clockwise(matrix):
    rows, cols = len(matrix), len(matrix[0])
    rotated = [[0] * rows for _ in range(cols)]

    for i in range(rows):
        for j in range(cols):
            rotated[j][rows - 1 - i] = matrix[i][j]

    return rotated

In [60]:
def rotate_90_counterclockwise(matrix):
    rows, cols = len(matrix), len(matrix[0])
    rotated = [[0] * rows for _ in range(cols)]

    for i in range(rows):
        for j in range(cols):
            rotated[cols - 1 - j][i] = matrix[i][j]

    return rotated

In [65]:
for q in range(400):

    n = len(arr)
    m = len(arr[0])

    for i in range(n):
        for j in range(m):
            x,y = i,j
            if arr[x][y] == "O":
                while x > 0 and arr[x-1][y] == ".":
                    arr[x][y], arr[x-1][y] = ".", "O"
                    x -= 1


    arr = rotate_90_clockwise(arr)

    res = 0
    for i in range(n):
        for j in range(m):
            if arr[i][j] == "O":
                res += n - i
    if q%4 == 3:
        print((q + 1) // 4, res)
    


1 102369
2 102338
3 102324
4 102329
5 102310
6 102303
7 102291
8 102279
9 102281
10 102282
11 102301
12 102341
13 102397
14 102455
15 102523
16 102588
17 102664
18 102718
19 102755
20 102812
21 102849
22 102893
23 102923
24 102956
25 102989
26 103003
27 102998
28 102979
29 102943
30 102905
31 102859
32 102810
33 102775
34 102747
35 102726
36 102696
37 102660
38 102625
39 102574
40 102522
41 102471
42 102417
43 102369
44 102338
45 102324
46 102329
47 102310
48 102303
49 102291
50 102279
51 102281
52 102282
53 102301
54 102341
55 102397
56 102455
57 102523
58 102588
59 102664
60 102718
61 102755
62 102812
63 102849
64 102893
65 102923
66 102956
67 102989
68 103003
69 102998
70 102979
71 102943
72 102905
73 102859
74 102810
75 102775
76 102747
77 102726
78 102696
79 102660
80 102625
81 102574
82 102522
83 102471
84 102417
85 102369
86 102338
87 102324
88 102329
89 102310
90 102303
91 102291
92 102279
93 102281
94 102282
95 102301
96 102341
97 102397
98 102455
99 102523
100 102588


In [62]:
1000000000 % 42

34

In [63]:
420 + 34

454